In [1]:
import sys
from pathlib import Path
import os
import gc
import datetime
from glob import glob
import numpy as np
import pandas as pd
import polars as pl

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

ROOT = '/kaggle/input/home-credit-credit-risk-model-stability'

In [2]:
start_time_utc = datetime.datetime.now()
print(f'Notebook Start Time (UTC): {start_time_utc}')

start_time_kst = start_time_utc + datetime.timedelta(hours=9)
print(f"Notebook Start Time (KST): {start_time_kst}")

Notebook Start Time (UTC): 2024-04-17 15:37:32.864485
Notebook Start Time (KST): 2024-04-18 00:37:32.864485


In [3]:
class Pipeline:

    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))
        return df

    def handle_dates(df):
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))  #!!?
                df = df.with_columns(pl.col(col).dt.total_days()) # t - t-1
        df = df.drop("date_decision", "MONTH")
        return df

    def filter_cols(df):
        for col in df.columns:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                isnull = df[col].is_null().mean()
                if isnull > 0.7:
                    df = df.drop(col)
        
        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.String):
                freq = df[col].n_unique()
                if (freq == 1) | (freq > 200):
                    df = df.drop(col)
        
        return df



class Aggregator:
    # Please add or subtract features yourself, be aware that too many features will take up too much space.
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        # expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        expr_median = [pl.median(col).alias(f"median_{col}") for col in cols]
        expr_var = [pl.var(col).alias(f"var_{col}") for col in cols]

        return expr_max + expr_last + expr_mean 

    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        # expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        # expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        expr_median = [pl.median(col).alias(f"median_{col}") for col in cols]

        return expr_max + expr_last + expr_mean 

    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        # expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        # expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        # expr_count = [pl.count(col).alias(f"count_{col}") for col in cols]
        return expr_max + expr_last  # +expr_count

    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        # expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        # expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return expr_max + expr_last

    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        # expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        # expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return expr_max + expr_last

    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

In [4]:
def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)
    if depth in [1,2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df)) 
    return df


def read_files(regex_path, depth=None):
    chunks = []
    
    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)
        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
        chunks.append(df)
    
    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])
    return df

In [5]:
def feature_eng(df_base, depth_0, depth_1, depth_2):
    df_base = (
        df_base
        .with_columns(
            month_decision = pl.col("date_decision").dt.month(),
            weekday_decision = pl.col("date_decision").dt.weekday(),
        )
    )
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
    df_base = df_base.pipe(Pipeline.handle_dates)
    return df_base


def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    return df_data, cat_cols

In [6]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type)=="category":
            continue
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            continue
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [7]:
%%time

ROOT            = Path(ROOT)

TRAIN_DIR       = ROOT / "parquet_files" / "train"
TEST_DIR        = ROOT / "parquet_files" / "test"

data_store = {
    "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        read_file(TRAIN_DIR / "train_static_cb_0.parquet"),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
        read_files(TRAIN_DIR / "train_credit_bureau_a_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_other_1.parquet", 1),
        read_file(TRAIN_DIR / "train_person_1.parquet", 1),
        read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
        read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
        read_files(TRAIN_DIR / "train_credit_bureau_a_2_*.parquet", 2),
        read_file(TRAIN_DIR / "train_applprev_2.parquet", 2),
        read_file(TRAIN_DIR / "train_person_2.parquet", 2)
    ]
}

CPU times: user 4min 49s, sys: 1min 38s, total: 6min 27s
Wall time: 2min 15s


In [8]:
%%time

df_train = feature_eng(**data_store)
print("train data shape:\t", df_train.shape)
del data_store
df_train = df_train.pipe(Pipeline.filter_cols)
gc.collect()

train data shape:	 (1526659, 861)
CPU times: user 37.2 s, sys: 12.3 s, total: 49.5 s
Wall time: 21 s


0

In [9]:
df_train, cat_cols = to_pandas(df_train)
df_train = reduce_mem_usage(df_train)
print("train data shape:\t", df_train.shape)
nums=df_train.select_dtypes(exclude='category').columns
from itertools import combinations, permutations
#df_train=df_train[nums]
nans_df = df_train[nums].isna()
nans_groups={}
for col in nums:
    cur_group = nans_df[col].sum()
    try:
        nans_groups[cur_group].append(col)
    except:
        nans_groups[cur_group]=[col]
del nans_df; x=gc.collect()

def reduce_group(grps):
    use = []
    for g in grps:
        mx = 0; vx = g[0]
        for gg in g:
            n = df_train[gg].nunique()
            if n>mx:
                mx = n
                vx = gg
            #print(str(gg)+'-'+str(n),', ',end='')
        use.append(vx)
        #print()
    print('Use these',use)
    return use

def group_columns_by_correlation(matrix, threshold=0.8):
    # 计算列之间的相关性
    correlation_matrix = matrix.corr()

    # 分组列
    groups = []
    remaining_cols = list(matrix.columns)
    while remaining_cols:
        col = remaining_cols.pop(0)
        group = [col]
        correlated_cols = [col]
        for c in remaining_cols:
            if correlation_matrix.loc[col, c] >= threshold:
                group.append(c)
                correlated_cols.append(c)
        groups.append(group)
        remaining_cols = [c for c in remaining_cols if c not in correlated_cols]
    
    return groups

uses=[]
for k,v in nans_groups.items():
    if len(v)>1:
            Vs = nans_groups[k]
            #cross_features=list(combinations(Vs, 2))
            #make_corr(Vs)
            grps= group_columns_by_correlation(df_train[Vs], threshold=0.8)
            use=reduce_group(grps)
            uses=uses+use
            #make_corr(use)
    else:
        uses=uses+v
    print('####### NAN count =',k)
print(uses)
print(len(uses))
uses=uses+list(df_train.select_dtypes(include='category').columns)
print(len(uses))
df_train=df_train[uses]
# df_train.drop(['requesttype_4525192L_cnt','max_empl_employedtotal_800L_cnt', 'max_empl_industry_691L_cnt'], axis=1, inplace=True)

Memory usage of dataframe is 4322.75 MB
Memory usage after optimization is: 1528.81 MB
Decreased by 64.6%
train data shape:	 (1526659, 472)
Use these ['case_id', 'WEEK_NUM', 'target', 'month_decision', 'weekday_decision', 'credamount_770A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_867L', 'clientscnt_1022L', 'clientscnt_100L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'deferredmnthsnum_166L', 'disbursedcredamount_1113A', 'downpmt_116A', 'homephncnt_628L', 'isbidproduct_1095L', 'mobilephncnt_593L', 'numactivecreds_622L', 'numactivecredschannel_414L', 'numactiverelcontr_750L', 'numcontrs3months_479L', 'numnotactivated_1143L', 'numpmtchanneldd_318L', 'numrejects9m_859L', 'sellerplacecnt_915L', 'max_mainoccupationinc_384A', 'max_birth_259D', 'max_num_group1_9']
####### NAN

In [10]:
y = df_train["target"]
weeks = df_train["WEEK_NUM"]
df_train= df_train.drop(columns=["target", "case_id", "WEEK_NUM"])

In [11]:
df_train[cat_cols] = df_train[cat_cols].astype(str)

In [12]:
from catboost import CatBoostClassifier, Pool

params = {
    "eval_metric": "AUC",  # 相当于LightGBM中的metric参数
    # "depth": 10,  # 相当于LightGBM中的max_depth参数
    "learning_rate": 0.03,
    "iterations":6000,  # 6000
    # "random_seed": 3107,  # 相当于LightGBM中的random_state参数
    # "l2_leaf_reg": 10,  # 相当于LightGBM中的reg_lambda参数
    # "border_count": 254,  # 没有直接相当于LightGBM中的colsample_by*参数，但可以用来增加分箱数
    "verbose": False,  # 控制输出信息
    "task_type": "GPU",  # 使用GPU训练
}

n_splits = 5
fitted_models = []
cv_scores = []

cv = StratifiedGroupKFold(n_splits=n_splits, shuffle=False)

step = 0
for idx_train, idx_valid in cv.split(df_train, y, groups=weeks):#   Because it takes a long time to divide the data set, 
    step += 1
    print(f'current step: {step}')
    
    X_train, y_train = df_train.iloc[idx_train], y.iloc[idx_train]# each time the data set is divided, two models are trained to each other twice, which saves time.
    X_valid, y_valid = df_train.iloc[idx_valid], y.iloc[idx_valid]

    train_pool = Pool(X_train, y_train,cat_features=cat_cols)
    val_pool = Pool(X_valid, y_valid,cat_features=cat_cols)

    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=val_pool, verbose=1000)

    
    fitted_models.append(model)
    y_pred_valid = model.predict_proba(X_valid)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores.append(auc_score)
    
print("CV AUC scores: ", cv_scores)
print("AVG CV AUC score: ", np.mean(cv_scores))
print("Maximum CV AUC score: ", max(cv_scores))

current step: 1


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6536968	best: 0.6536968 (0)	total: 14.8s	remaining: 1d 40m 55s
1000:	test: 0.8399774	best: 0.8399774 (1000)	total: 12m 45s	remaining: 1h 3m 40s
2000:	test: 0.8431761	best: 0.8431761 (2000)	total: 24m 52s	remaining: 49m 43s
3000:	test: 0.8450022	best: 0.8450022 (3000)	total: 36m 58s	remaining: 36m 57s
4000:	test: 0.8462389	best: 0.8462389 (4000)	total: 49m 3s	remaining: 24m 30s
5000:	test: 0.8471753	best: 0.8471753 (5000)	total: 1h 1m 7s	remaining: 12m 12s
5999:	test: 0.8478975	best: 0.8478975 (5999)	total: 1h 13m 14s	remaining: 0us
bestTest = 0.8478975296
bestIteration = 5999
current step: 2


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6542186	best: 0.6542186 (0)	total: 782ms	remaining: 1h 18m 11s
1000:	test: 0.8420379	best: 0.8420379 (1000)	total: 12m 40s	remaining: 1h 3m 18s
2000:	test: 0.8456873	best: 0.8456905 (1990)	total: 24m 56s	remaining: 49m 50s
3000:	test: 0.8472965	best: 0.8472965 (3000)	total: 37m 4s	remaining: 37m 2s
4000:	test: 0.8485040	best: 0.8485042 (3995)	total: 49m 15s	remaining: 24m 36s
5000:	test: 0.8492461	best: 0.8492461 (5000)	total: 1h 1m 26s	remaining: 12m 16s
5999:	test: 0.8497642	best: 0.8497642 (5999)	total: 1h 13m 35s	remaining: 0us
bestTest = 0.8497641683
bestIteration = 5999
current step: 3


...

0:	test: 0.6591796	best: 0.6591796 (0)	total: 783ms	remaining: 1h 18m 14s
1000:	test: 0.8465410	best: 0.8465410 (1000)	total: 12m 36s	remaining: 1h 2m 57s
2000:	test: 0.8498071	best: 0.8498071 (2000)	total: 24m 43s	remaining: 49m 25s
3000:	test: 0.8515902	best: 0.8515910 (2995)	total: 36m 52s	remaining: 36m 50s
4000:	test: 0.8527260	best: 0.8527260 (4000)	total: 49m 1s	remaining: 24m 29s
5000:	test: 0.8536931	best: 0.8536931 (5000)	total: 1h 1m 9s	remaining: 12m 13s
5999:	test: 0.8543816	best: 0.8543816 (5999)	total: 1h 13m 18s	remaining: 0us
bestTest = 0.8543815613
bestIteration = 5999
current step: 4


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6577309	best: 0.6577309 (0)	total: 783ms	remaining: 1h 18m 17s
1000:	test: 0.8450685	best: 0.8450685 (1000)	total: 12m 36s	remaining: 1h 2m 55s
2000:	test: 0.8482993	best: 0.8482993 (2000)	total: 24m 50s	remaining: 49m 38s
3000:	test: 0.8500742	best: 0.8500742 (3000)	total: 36m 57s	remaining: 36m 55s
4000:	test: 0.8512815	best: 0.8512815 (4000)	total: 49m 11s	remaining: 24m 34s
5000:	test: 0.8520714	best: 0.8520714 (5000)	total: 1h 1m 19s	remaining: 12m 15s
5999:	test: 0.8527253	best: 0.8527253 (5999)	total: 1h 13m 28s	remaining: 0us
bestTest = 0.8527252674
bestIteration = 5999
current step: 5


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6502633	best: 0.6502633 (0)	total: 780ms	remaining: 1h 18m 1s
1000:	test: 0.8412796	best: 0.8412796 (1000)	total: 12m 36s	remaining: 1h 2m 58s
2000:	test: 0.8448407	best: 0.8448407 (2000)	total: 24m 49s	remaining: 49m 36s
3000:	test: 0.8466499	best: 0.8466522 (2995)	total: 37m	remaining: 36m 59s
4000:	test: 0.8477974	best: 0.8477974 (4000)	total: 49m 13s	remaining: 24m 35s
5000:	test: 0.8486795	best: 0.8486829 (4995)	total: 1h 1m 25s	remaining: 12m 16s
5999:	test: 0.8493560	best: 0.8493560 (5999)	total: 1h 13m 38s	remaining: 0us
bestTest = 0.8493559659
bestIteration = 5999
CV AUC scores:  [0.847897544034846, 0.8497641600464011, 0.8543815801618762, 0.8527252280285931, 0.8493560272819478]
AVG CV AUC score:  0.8508249079107328
Maximum CV AUC score:  0.8543815801618762


In [13]:
import joblib

joblib.dump(fitted_models, 'cat_models.joblib')

notebook_info = {
    'notebook_start_time': start_time_kst,
    'description': 'first cat models',
    'cols': df_train.columns.to_list(),
    'cat_cols': cat_cols,
}
joblib.dump(notebook_info, 'notebook_info.joblib')


['notebook_info.joblib']

In [14]:
!ls -al

total 150664
drwxr-xr-x 3 root root      4096 Apr 17 22:12 .
drwxr-xr-x 6 root root      4096 Apr 17 15:37 ..
---------- 1 root root     58787 Apr 17 22:12 __notebook__.ipynb
-rw-r--r-- 1 root root 154193424 Apr 17 22:12 cat_models.joblib
drwxr-xr-x 5 root root      4096 Apr 17 15:44 catboost_info
-rw-r--r-- 1 root root     10455 Apr 17 22:12 notebook_info.joblib
